In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math
from matplotlib.lines import Line2D
import os
import json
import matplotlib.ticker as ticker
import seaborn as sns
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from scipy.interpolate import make_interp_spline, BSpline
os.chdir("C:/Users/Zber/Documents/Dev_program/OpenRadar")
from FER.utils import MapRecord
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch

In [26]:
class ImageNet_Small_v1(nn.Module):
    def __init__(self, num_channel=1):
        super(ImageNet_Small_v1, self).__init__()

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.group1 = nn.Sequential(
            nn.Conv2d(num_channel, 16, kernel_size=(3, 3), stride=1, padding=(1, 1)),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d((1, 1)))

        self.group2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=(5, 3), stride=1, padding=(1, 1)),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d((2, 2)))

        self.group3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=(5, 3), stride=1, padding=(1, 1)),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d((2, 1)),
        )

        self.group4 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=1, padding=(2, 1)),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d((2, 1)),
        )


    def forward(self, x):
        x = self.group1(x)
        g1 = self.group2(x)
        g2 = self.group3(g1)
        g3 = self.group4(g2)
        return x, g1, g2, g3


In [ ]:
class TFblock_v4(nn.Module):
    def __init__(self, dim_in=512, dim_inter=1024):
        super(TFblock_v4, self).__init__()
        # self.encoder = nn.Linear(dim_in, dim_inter)
        # self.decoder = nn.Linear(dim_inter, dim_in)

        self.encoder = nn.Conv2d(in_channels=dim_in, out_channels=dim_inter, kernel_size=(4, 2))
        self.bn1 = nn.BatchNorm2d(num_features=dim_inter)
        self.decoder = nn.ConvTranspose2d(in_channels=dim_inter, out_channels=(dim_in + dim_inter) // 2, kernel_size=(
            3, 3), stride=(1, 1), padding=(0, 0))
        self.decoder1 = nn.ConvTranspose2d(in_channels=(dim_in + dim_inter) // 2, out_channels=dim_in, kernel_size=(
            5, 5), stride=(1, 1), padding=(0, 0))

    def forward(self, x):
        x = self.encoder(x)
        # x = self.avgpool(x)
        x = self.bn1(x)
        x = self.decoder(x)
        x = self.decoder1(x)
        return x

In [31]:
class TFblock_v3(nn.Module):
    def __init__(self, dim_in=512, dim_inter=1024):
        super(TFblock_v3, self).__init__()
        # self.encoder = nn.Linear(dim_in, dim_inter)
        # self.decoder = nn.Linear(dim_inter, dim_in)

        self.encoder = nn.Conv2d(dim_in, dim_inter, (2, 2))
        self.bn1 = nn.BatchNorm2d(num_features=dim_inter)
        self.decoder = nn.ConvTranspose2d(in_channels=dim_inter, out_channels=dim_in, kernel_size=(
            3, 3), stride=(1, 1), padding=(0, 0))

    def forward(self, x):
        x = self.encoder(x)
        # x = self.avgpool(x)
        x = self.bn1(x)
        x = self.decoder(x)
        return x


In [4]:
m = nn.AdaptiveAvgPool2d((5, 7))
input = torch.randn(1, 64, 8, 9)
output = m(input)
print(output.size())


torch.Size([1, 64, 5, 7])


In [2]:
a = torch.randn(2, 3, 3)
a.size()

torch.Size([2, 3, 3])

In [3]:
a = a.unsqueeze(3)
a.size()

torch.Size([2, 3, 3, 1])

In [10]:
a = torch.randn(16, 512)
b = torch.randn(16, 512)

c = torch.cdist(a, b, p=2, compute_mode="donot_use_mm_for_euclid_dist")
c.size()


torch.Size([16, 16])

In [17]:
a = np.zeros((16, 512))
b = np.ones((16, 512))

c = np.linalg.norm(a - b)
print(c)


90.50966799187809
